**Paso 6: Implementación específica de la capa RAG**


In [ ]:
# Instalar la biblioteca principal de LangChain
! pip install langchain

# Instalar FAISS (necesario para el vector store que estás usando)
! pip install faiss-cpu

# Opcional pero recomendado: si usas Google Generative AI (ya lo habías instalado, pero lo reconfirmo)
! pip install langchain-google-genai

In [ ]:
# Importar librerías necesarias para RAG con Gemini
# Asegúrate de instalar: pip install langchain-google-genai
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA

# Añadir más textos relevantesAPI para Gemini
# Cargar documentos locales o datasets que forman el "mini Data-Lake"
# Ejemplo: Una lista con textos de normativas, guías agrícolas y financieras
documents = [
    "Normativa Agua Cuenca Segura 2025: Restringe riego, promueve eficiencia hídrica.",
    "Informe Mercado Agrícola Murcia 2024: Volatilidad precios, demanda creciente.",
    "Guía Créditos Agrícolas."
]

# Crear embeddings para los documentos
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001") # Modelo de embedding de Gemini

# Crear base vectorial FAISS
vectorstore = FAISS.from_texts(documents, embeddings)

#  Crear el LLM de Gemini para la cadena RAG
gemini_llm_rag = ChatGoogleGenerativeAI(model="gemini-2.5-flash") # El modelo de chat para LangChain

# Crear cadena RAG para preguntas y respuestas con contexto documental
# Crear cadena RAG (usando el LLM de Gemini)
retriever = vectorstore.as_retriever()
rag_chain = RetrievalQA.from_chain_type(
    llm=gemini_llm_rag, 
    chain_type="stuff", 
    retriever=retriever)

# Función para que el agente coordinador use la capa RAG y enriquezca respuestas
def rag_coordinator_query(query_text):
    """
    Realiza consulta a capa RAG para enriquecer recomendaciones con respaldo documental.
    """
    result = rag_chain.invoke(query_text) # Uso de .invoke() más moderno en LangChain
    return  ['result']

# Ejemplo de uso dentro del flujo multiagente:
query_example = "¿Cuáles son las restricciones actuales para el uso de agua en la cuenca del Segura?"
rag_response = rag_coordinator_query(query_example)

print("Respuesta enriquecida con capa RAG:")
print(rag_response)

# En el sistema completo, esta lógica se integraría en la ejecución del agente 'rag_coordinator',
# quien consultaría el mini Data-Lake antes de generar la respuesta final que respalde
# las recomendaciones financieras y agronómicas para el agricultor.

**Paso 7:  Respuestas simuladas o ejemplos de outputs por agente (opcional para pruebas y demostración)**

In [ ]:
# Agente: Investigador de Mercado
# Tarea:
# Analiza problemas y condiciones del mercado agrícola en Murcia.

# Respuesta final:

market_researcher_answer = """
**Problemas Clave de Agricultores en la Región de Murcia**

Esta es una lista detallada de problemas que enfrentan los agricultores locales, respaldados con datos cuantificables cuando se dispone:

1. Variabilidad y volatilidad de precios agrícolas:
   - Problema: Los precios de cultivos como la uva y el cítrico presentan alta fluctuación, dificultando la planificación financiera.
   - Datos: En los últimos 5 años, la volatilidad de precios ha aumentado un 20%, afectando los ingresos medios por hectárea.

2. Limitado acceso a créditos y financiamiento inclusivo:
   - Problema: Muchos pequeños agricultores tienen dificultades para acceder a créditos bancarios formales debido a requisitos estrictos.
   - Datos: Solo el 35% de pequeños agricultores tienen acceso a microcréditos o subsidios regionales.

3. Escasez y gestión de agua en la cuenca del Segura:
   - Problema: Restricciones en el uso de recursos hídricos afectan la productividad y elección de cultivos.
   - Datos: Restricciones legales han reducido el agua disponible para riego un 15% en los últimos 3 años.

4. Impacto del cambio climático y estrés por variabilidad climática:
   - Problema: Cambios en patrones de temperatura y lluvias generan pérdidas de producción.
   - Datos: Se estima una reducción media del 10% en rendimiento agrícola atribuida a variabilidad climática.

5. Costos crecientes de insumos agrícolas:
   - Problema: Incrementos en precios de fertilizantes, pesticidas y energía impactan directamente los márgenes.
   - Datos: Precios de insumos subieron un 12% en el último año.
"""

# Agente: Especialista Agronómico
# Tarea:
# Proporciona recomendaciones técnicas sobre cultivos y clima locales.

agronomic_expert_answer = """
**Recomendaciones Agronómicas para la Región de Murcia**

- Cultivos recomendados: uva, olivo, almendra, adaptados a las condiciones mediterráneas.
- Gestión recomendada del suelo: rotación con leguminosas para mejorar nitrógeno.
- Optimización de riego: implementar sistemas de riego por goteo y monitorización de humedad.
- Uso de variedades resistentes a estrés hídrico y calor.
"""

# Agente: Asesor Financiero Inclusivo
# Tarea:
# Recomienda opciones financieras claras y accesibles para agricultores.

financial_advisor_answer = """
**Opciones Financieras para Pequeños Agricultores**

- Microcréditos con tasas del 5-7% accesibles para pequeños productores.
- Subsidios regionales con condiciones flexibles para proyectos sostenibles.
- Programas de financiamiento colectivo (crowdfunding agrícola).
- Capacitación financiera para mejorar el acceso al crédito formal.
"""

# Agente: Explicador Inclusivo
# Tarea:
# Transforma recomendaciones técnicas y financieras en lenguaje claro.

inclusive_explainer_answer = """
**Explicaciones Claras para Agricultores**

"Los precios de los cultivos pueden cambiar mucho. Para evitar problemas financieros recomendamos solicitar pequeños créditos que son fáciles de obtener y ayudan a comprar insumos. También sugerimos usar técnicas para ahorrar agua y escoger cultivos que pueden resistir mejor el calor. Todo esto ayuda a que tu cosecha sea más segura y rentable."
"""

# Agente: Coordinador con Capa RAG
# Tarea:
# Recupera y resume documentos normativos y técnicos que respaldan las recomendaciones.

rag_coordinator_answer = """
**Documentos Consultados**

- Normativa Agua Cuenca Segura 2025 - Resumen: Restricciones actuales, recomendaciones sobre riego.
- Informe de Mercado Agrícola Murcia 2024 - Resumen: Tendencias y volatilidad de precios.
- Guía Oficial de Créditos Agrícolas Inclusivos - Resumen: Requisitos, tasas y beneficios.
"""

# Ejemplo para mostrar todas las respuestas juntas
print("=== Investigador de Mercado ===")
print(market_researcher_answer)
print("\n=== Especialista Agronómico ===")
print(agronomic_expert_answer)
print("\n=== Asesor Financiero Inclusivo ===")
print(financial_advisor_answer)
print("\n=== Explicador Inclusivo ===")
print(inclusive_explainer_answer)
print("\n=== Coordinador con Capa RAG ===")
print(rag_coordinator_answer)
